In [19]:
import csv
import pandas
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline 
import time
from datetime import datetime
import numpy as np
import seaborn as sns
import nltk 
from collections import Counter

In [2]:
# lump together all data
results_1 = pandas.DataFrame.from_csv('../SURVEY/RESULTS/DATA/flattened_all_with_text.csv', encoding='utf8') 
print len(results_1)
results_2 = pandas.DataFrame.from_csv('../STUDY2/DATA/all_results/flattened_title_text.csv',encoding='utf8')
print len(results_2)
all_data = pandas.concat([results_1, results_2])
print len(all_data)

360
1280
1640


In [3]:
name_to_last = {'Bernie Sanders':'sanders', 'Donald Trump':'trump', 
                'Hillary Clinton':'clinton', 'Ted Cruz':'cruz'}
all_data['candidate'] = all_data.candidate.apply(lambda row: name_to_last[row] if row in name_to_last.keys() else row)

In [4]:
all_data.drop(["is_complex","flesch","gunning_fog","readability"], axis=1, inplace=True)

In [5]:
print "COLUMNS:"
print "\t".join(list(all_data.columns))
print "TOTAL ANNOTATIONS:", len(all_data)
print "TOTAL NUMBER OF STORIES:", len(set(all_data.title))
print "TOTAL NUMBER PER CANDIDATE:"
all_data.groupby('candidate').title.count()

COLUMNS:
age	body	candidate	fair	favor	gender	org	party	source	state	title	topic	trust	url	voting_for
TOTAL ANNOTATIONS: 1640
TOTAL NUMBER OF STORIES: 125
TOTAL NUMBER PER CANDIDATE:


candidate
clinton    410
cruz       410
sanders    410
trump      410
Name: title, dtype: int64

In [6]:
# we apparently have some repeat stories. it's ok.
print len(set(all_data.body))
print len(set(all_data.title))

122
125


## FIRST SHOT IN THE DARK: BIGRAMS

In [15]:
import mutualInformation
import wordportrait

"""
wordportrait.miportrait(
 Inputs:
    userstat - dictionary of counts of words in tweets of target user (or group)
    jointstat - dictionary of counts of words in everyone tweets
    verbose - if True, adds frequencies and counts to the output (useful for debugging)
    
    Returns:
        - The list of pairs (word, mutual information) if verbose = False
        - The list of tuples (word, frequency by user, frequency by everyone else, count by user, count by everyone else, mutual information) if verbose = True
   
"""

In [16]:
all_data['tokens'] = all_data.body.apply(lambda row: nltk.word_tokenize(row))

In [20]:
all_data['word_count'] = all_data.tokens.apply(lambda row: Counter(row))

In [25]:
all_counts = all_data.word_count.sum()

In [28]:
all_counts.most_common(100)

[(u',', 36220),
 (u'the', 30316),
 (u'.', 27849),
 (u'to', 18390),
 (u'a', 17814),
 (u'and', 15942),
 (u'of', 15070),
 (u'in', 14139),
 (u"''", 9783),
 (u'``', 9762),
 (u"'s", 8865),
 (u'that', 7829),
 (u'Trump', 7767),
 (u'his', 7317),
 (u'he', 7227),
 (u'is', 6652),
 (u'on', 6162),
 (u'said', 5865),
 (u'for', 5711),
 (u'has', 5292),
 (u'Cruz', 4923),
 (u'at', 4706),
 (u'as', 4700),
 (u'Iowa', 4684),
 (u'with', 4480),
 (u'be', 4299),
 (u'Sanders', 4053),
 (u'The', 4023),
 (u'who', 4008),
 (u'from', 3936),
 (u'campaign', 3896),
 (u'Clinton', 3295),
 (u'have', 2736),
 (u'will', 2719),
 (u'by', 2605),
 (u'I', 2592),
 (u'it', 2551),
 (u'\u2014', 2546),
 (u'was', 2529),
 (u'are', 2494),
 (u'presidential', 2479),
 (u'not', 2478),
 (u'Mr.', 2433),
 (u'would', 2381),
 (u'voters', 2316),
 (u'New', 2233),
 (u'an', 2192),
 (u'conservative', 2079),
 (u':', 2027),
 (u'group', 1873),
 (u'do', 1800),
 (u'Democratic', 1773),
 (u'this', 1723),
 (u"n't", 1701),
 (u'candidate', 1667),
 (u'president', 15

In [50]:
# remove stopwords
from nltk.corpus import stopwords
import string
punct = [x for x in string.punctuation]
skip = stopwords.words('english') + punct + ["''", '``',"'s"]

In [51]:
all_data['filtered_tokens'] = all_data.tokens.apply(lambda row: [word for word in row if word not in skip])
#filtered_words = [word for word in word_list if word not in stopwords.words('english')]

In [52]:
all_data['filtered_word_count'] = all_data.filtered_tokens.apply(lambda row: Counter(row))

In [53]:
all_filtered_counts = all_data.filtered_word_count.sum()

In [55]:
all_filtered_counts.most_common(20)

[(u'Trump', 7767),
 (u'said', 5865),
 (u'Cruz', 4923),
 (u'Iowa', 4684),
 (u'Sanders', 4053),
 (u'The', 4023),
 (u'campaign', 3896),
 (u'Clinton', 3295),
 (u'I', 2592),
 (u'\u2014', 2546),
 (u'presidential', 2479),
 (u'Mr.', 2433),
 (u'would', 2381),
 (u'voters', 2316),
 (u'New', 2233),
 (u'conservative', 2079),
 (u'group', 1873),
 (u'Democratic', 1773),
 (u"n't", 1701),
 (u'candidate', 1667)]